In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

import time
import os
import glob
import random
import json
import subprocess
import sys
import gc

import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import xml.etree.ElementTree

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.decomposition import PCA
def swap(i,j):
  a = i.swapaxes(0,1)
  b = j.swapaxes(0,1)
  return a,b

def unsqueeze(i,j):
  a = torch.unsqueeze(i,0)
  b = torch.unsqueeze(j,0)
  return a,b
def set_deterministic():
    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
    torch.set_deterministic(True)
    
    
def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
def compute_test_accuracy(Rai, Raf, ti, tf, mode, TRAIN_SIZE, BATCH_SIZE, SEED, var, model, device):

  data_x, data_y = load_data(Rai, Raf, ti, tf, mode)

  image_train_loader, image_test_loader = generate_image_loader(data_x, data_y, TRAIN_SIZE, BATCH_SIZE, SEED)  

  data_x, data_y, n_components_x, n_components_y, pca_x, pca_y = principal_components(data_x, data_y, var)

  train_loader, test_loader = generate_loader(data_x, data_y, TRAIN_SIZE, BATCH_SIZE, SEED)

  model.eval()
  numers = []
  denoms = []

  refs_list = []

  with torch.no_grad():
  
    for i, (_, refs) in enumerate(image_test_loader):

      refs = refs.to(device)

      refs = np.array(refs.cpu())

      refs = np.reshape(refs, (np.shape(refs)[0], -1))

      refs_list = refs_list + [refs]


  with torch.no_grad():
  
    for i, (features, targets) in enumerate(test_loader):

      print(np.shape(features))

      features = features.to(device)

      pred = model(features)

      pred = np.array(pred.cpu())
      references = refs_list[i]

      pred = pca_y.inverse_transform(pred)

      numer = np.sum(np.square(np.array(references) - np.array(pred)))
      numers = numers + [numer]

      denom = np.sum(np.square(np.array(references)))
      denoms = denoms + [denom]


  acc = 1 - np.array(numers)/np.array(denoms)

  return np.round(acc[0], 5)

def compute_test_accuracy2(Rai, Raf, ti, tf, TRAIN_SIZE, BATCH_SIZE, SEED, var, model_diff, model_conv, device):

  data_x, data_y = load_data(Rai, Raf, ti, tf, mode = 'no_masking')
  image_train_loader, image_test_loader = generate_image_loader(data_x, data_y, TRAIN_SIZE, BATCH_SIZE, SEED)  

  data_x_diff, data_y_diff = load_data(Rai, Raf, ti, tf, mode = 'diffusion_mask')
  data_x_diff, data_y_diff, n_components_x_diff, n_components_y_diff, pca_x_diff, pca_y_diff = principal_components(data_x_diff, data_y_diff, var)
  train_loader_diff, test_loader_diff = generate_loader(data_x_diff, data_y_diff, TRAIN_SIZE, BATCH_SIZE, SEED)

  data_x_conv, data_y_conv = load_data(Rai, Raf, ti, tf, mode = 'convection_mask')
  data_x_conv, data_y_conv, n_components_x_conv, n_components_y_conv, pca_x_conv, pca_y_conv = principal_components(data_x_conv, data_y_conv, var)
  train_loader_conv, test_loader_conv = generate_loader(data_x_conv, data_y_conv, TRAIN_SIZE, BATCH_SIZE, SEED)


  numers = []
  denoms = []

  refs_list = []

  with torch.no_grad():
  
    for i, (_, refs) in enumerate(image_test_loader):

      print(i)

      refs = refs.to(device)

      refs = np.array(refs.cpu())

      refs = np.reshape(refs, (np.shape(refs)[0], -1))

      print(np.shape(refs))

      refs_list = refs_list + [refs]

  #print(np.shape(refs_list[0]),np.shape(refs_list[1]),np.shape(refs_list[2]))


  with torch.no_grad():

    p = 0

    for (features_diff, _), (features_conv, _) in zip(test_loader_diff, test_loader_conv):

      #print(np.shape(refs_list[0]),np.shape(refs_list[1]),np.shape(refs_list[2]))

      features_diff = features_diff.to(device)
      features_conv = features_conv.to(device)

      print(np.shape(features_diff))
      print(np.shape(features_conv))

      pred_diff = model_diff(features_diff) 
      pred_conv = model_conv(features_conv)

      print(np.shape(pred_diff))
      print(np.shape(pred_conv))

      pred_diff = np.array(pred_diff.cpu())
      pred_conv = np.array(pred_conv.cpu())

      print(np.shape(pred_diff))
      print(np.shape(pred_conv))

      #references = refs_list[p]

      print(np.shape(refs_list[p]))

      pred_diff = pca_y_diff.inverse_transform(pred_diff)

      pred_conv = pca_y_conv.inverse_transform(pred_conv)

      pred = pred_diff + pred_conv

      print(np.shape(pred))

      numer = np.sum(np.square(np.array(refs_list[p]) - np.array(pred)))
      numers = numers + [numer]

      denom = np.sum(np.square(np.array(refs_list[p])))
      denoms = denoms + [denom]

      p = p + 1

  acc = 1 - np.array(numers)/np.array(denoms)

  return np.round(acc[0], 5)


def compute_epoch_loss_autoencoder(model, data_loader, loss_fn, device):
  model.eval()
  curr_loss, num_examples = 0., 0
  with torch.no_grad():
    for features, targets in data_loader:

      features = features.to(device)
      targets = targets.to(device)

      predictions = model(features)
      loss = loss_fn(predictions, targets, reduction='sum')
      num_examples += targets.size(0)
      curr_loss += loss

      features = features.to('cpu')
      targets = targets.to('cpu')

      del features
      del targets

  curr_loss = curr_loss / num_examples

  return curr_loss
def train_autoencoder_v1(num_epochs, model, optimizer, 
                         train_loader, device, loss_fn=None, 
                         skip_epoch_stats=False,
                         save_model=None):
    
    log_dict = {'train_loss_per_batch': [],
                'train_loss_per_epoch': []}
    
    if loss_fn is None:
        loss_fn = F.mse_loss

    start_time = time.time()
    for epoch in range(num_epochs):

        model.train()
        for batch_idx, (features, targets) in enumerate(train_loader):

            features = features.to(device)
            targets = targets.to(device)

            # FORWARD AND BACK PROP
            predictions = model(features)
            loss = loss_fn(predictions, targets)
            optimizer.zero_grad()

            #loss.requires_grad = True

            loss.backward()

            # UPDATE MODEL PARAMETERS
            optimizer.step()

            #additional

            features = features.to('cpu')
            targets = targets.to('cpu')

            del features
            del targets

            # LOGGING
            log_dict['train_loss_per_batch'].append(loss.item())
            
            print('Epoch: %03d/%03d | Batch %04d/%04d | Loss: %.4f'
                  % (epoch+1, num_epochs, batch_idx+1,
                       len(train_loader), loss))

        if not skip_epoch_stats:
            model.eval()
            
            with torch.set_grad_enabled(False):  # save memory during inference
                
                train_loss = compute_epoch_loss_autoencoder(
                    model, train_loader, loss_fn, device)
                print('***Epoch: %03d/%03d | Loss: %.3f' % (
                      epoch+1, num_epochs, train_loss))
                log_dict['train_loss_per_epoch'].append(train_loss.item())

        print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

    print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))
    if save_model is not None:
        torch.save(model.state_dict(), save_model)
    
    return log_dict
def plot_training_loss(minibatch_losses, num_epochs, averaging_iterations=100, custom_label=''):

    iter_per_epoch = len(minibatch_losses) // num_epochs

    plt.figure()
    ax1 = plt.subplot(1, 1, 1)
    ax1.plot(range(len(minibatch_losses)),
             (minibatch_losses), label=f'Minibatch Loss{custom_label}')
    ax1.set_xlabel('Iterations')
    ax1.set_ylabel('Loss')

    if len(minibatch_losses) < 1000:
        num_losses = len(minibatch_losses) // 2
    else:
      num_losses = 1000

    #ax1.set_ylim([
    #    0, np.max(minibatch_losses[num_losses:])*1.5
    #    ])

    ax1.plot(np.convolve(minibatch_losses,
                         np.ones(averaging_iterations,)/averaging_iterations,
                         mode='valid'),
             label=f'Running Average{custom_label}')
    ax1.legend()

def plot_accuracy(train_acc, valid_acc):

    num_epochs = len(train_acc)

    plt.plot(np.arange(1, num_epochs+1), 
             train_acc, label='Training')
    plt.plot(np.arange(1, num_epochs+1),
             valid_acc, label='Validation')

    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
def load_data(Rai, Raf, ti, tf, mode):

  file_count = 0

  if mode == 'no_masking':

    _, _, files = next(os.walk('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_13_1_19'  + '/snapshots/' + str(ti) + '/arrays'))
    file_count = len(files)

  elif mode == 'diffusion_mask':

    _, _, files = next(os.walk('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_13_1_19'  + '/masks/diffusion/' + str(ti) + '/arrays'))
    file_count = len(files)

  elif mode == 'convection_mask':

    _, _, files = next(os.walk('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_13_1_19'  + '/masks/convection/' + str(ti) + '/arrays'))
    file_count = len(files)


  data_x = []
  file_list_x = []

  data_y = []
  file_list_y = []


  for i in range(file_count):

    if i > 2140: 
      continue  

    if mode == 'no_masking':

      file_e = glob.glob('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + '13_1_19' + '/snapshots/' + str(ti) + '/arrays/' + str(i) + '_' + '*.npy')
      file_list_x = file_list_x + [file_e]

    elif mode == 'diffusion_mask':
      file_e = glob.glob('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + '13_1_19' + '/masks/diffusion/' + str(ti) + '/arrays/' + str(i) + '_' + '*.npy')
      file_list_x = file_list_x + [file_e]

    elif mode == 'convection_mask':
      file_e = glob.glob('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + '13_1_19' + '/masks/convection/' + str(ti) + '/arrays/' + str(i) + '_' + '*.npy')
      file_list_x = file_list_x + [file_e]

  file_list_x = np.ravel(file_list_x)

  for file_path in file_list_x:
    
    a = np.load(file_path, allow_pickle=True)

    data_x = data_x + [a]

  for i in range(file_count):

    if i > 2140: 
      continue 

    if mode == 'no_masking':

      file_e = glob.glob('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + '13_1_19'  + '/snapshots/' + str(tf) + '/arrays/' + str(i) + '_' + '*.npy')
      file_list_y = file_list_y + [file_e]

    elif mode == 'diffusion_mask':

      file_e = glob.glob('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + '13_1_19'  + '/masks/diffusion/' + str(tf) + '/arrays/' + str(i) + '_' + '*.npy')
      file_list_y = file_list_y + [file_e]
  
    elif mode == 'convection_mask':

      file_e = glob.glob('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + '13_1_19'  + '/masks/convection/' + str(tf) + '/arrays/' + str(i) + '_' + '*.npy')
      file_list_y = file_list_y + [file_e]

  file_list_y = np.ravel(file_list_y)

  for file_path in file_list_y:
    
    a = np.load(file_path, allow_pickle=True)

    data_y = data_y + [a]
  
  return data_x, data_y
def print_files(Rai, Raf, ti, tf, mode):

  file_count = 0

  if mode == 'no_masking':

    _, _, files = next(os.walk('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_13_1_19'  + '/snapshots/' + str(ti) + '/arrays'))
    file_count = len(files)

  elif mode == 'diffusion_mask':

    _, _, files = next(os.walk('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_13_1_19'  + '/masks/diffusion/' + str(ti) + '/arrays'))
    file_count = len(files)

  elif mode == 'convection_mask':

    _, _, files = next(os.walk('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_13_1_19'  + '/masks/convection/' + str(ti) + '/arrays'))
    file_count = len(files)


  data_x = []
  file_list_x = []

  data_y = []
  file_list_y = []


  for i in range(file_count):

    if i > 2140: 
      continue  

    if mode == 'no_masking':

      file_e = glob.glob('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + '13_1_19' + '/snapshots/' + str(ti) + '/arrays/' + str(i) + '_' + '*.npy')
      file_list_x = file_list_x + [file_e]

    elif mode == 'diffusion_mask':
      file_e = glob.glob('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + '13_1_19' + '/masks/diffusion/' + str(ti) + '/arrays/' + str(i) + '_' + '*.npy')
      file_list_x = file_list_x + [file_e]

    elif mode == 'convection_mask':
      file_e = glob.glob('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + '13_1_19' + '/masks/convection/' + str(ti) + '/arrays/' + str(i) + '_' + '*.npy')
      file_list_x = file_list_x + [file_e]

  file_list_x = np.ravel(file_list_x)


  for i in range(file_count):

    if i > 2140: 
      continue 

    if mode == 'no_masking':

      file_e = glob.glob('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + '13_1_19'  + '/snapshots/' + str(tf) + '/arrays/' + str(i) + '_' + '*.npy')
      file_list_y = file_list_y + [file_e]

    elif mode == 'diffusion_mask':

      file_e = glob.glob('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + '13_1_19'  + '/masks/diffusion/' + str(tf) + '/arrays/' + str(i) + '_' + '*.npy')
      file_list_y = file_list_y + [file_e]
  
    elif mode == 'convection_mask':

      file_e = glob.glob('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + '13_1_19'  + '/masks/convection/' + str(tf) + '/arrays/' + str(i) + '_' + '*.npy')
      file_list_y = file_list_y + [file_e]

  file_list_y = np.ravel(file_list_y)

  
  return file_list_x, file_list_y
def split_files(file_list_x, file_list_y, train_size, seed):


  file_list_x = random.Random(seed).sample(list(file_list_x), len(file_list_x))
  file_list_y = random.Random(seed).sample(list(file_list_y), len(file_list_y))


  middle_index = int(np.round(len(file_list_x) * train_size))

  file_list_x_train = file_list_x[:middle_index]
  file_list_x_test = file_list_x[middle_index:]

  file_list_y_train = file_list_y[:middle_index]
  file_list_y_test = file_list_y[middle_index:]

  return file_list_x_train, file_list_y_train, file_list_x_test, file_list_y_test
def principal_components(data_x, data_y, var = 0.95):

  data_x = np.reshape(data_x, (np.shape(data_x)[0], -1))
  data_y = np.reshape(data_y, (np.shape(data_y)[0], -1))

  pca_x = PCA(var)
  pca_y = PCA(var)

  data_x = pca_x.fit_transform(data_x)
  n_components_x = pca_x.n_components_

  data_y = pca_y.fit_transform(data_y)
  n_components_y = pca_y.n_components_


  return data_x.tolist(), data_y.tolist(), n_components_x, n_components_y, pca_x, pca_y
def split_x_y_list(data_x, data_y, train_size, seed):


  data_x = random.Random(seed).sample(data_x, len(data_x))
  data_y = random.Random(seed).sample(data_y, len(data_y))


  middle_index = int(np.round(len(data_x) * train_size))

  data_x_train = data_x[:middle_index]
  data_x_test = data_x[middle_index:]

  data_y_train = data_y[:middle_index]
  data_y_test = data_y[middle_index:]

  return data_x_train, data_y_train, data_x_test, data_y_test
def generate_loader(data_x, data_y, TRAIN_SIZE, BATCH_SIZE, SEED):

  data_x_train, data_y_train, data_x_test, data_y_test = split_x_y_list(data_x, data_y, TRAIN_SIZE, SEED)

  ##########################
  ### Dataset
  ##########################

  import torch
  import numpy as np
  from torch.utils.data import TensorDataset, DataLoader

  import random

  my_x_train = np.array(data_x_train)
  my_x_test = np.array(data_x_test)

  my_y_train = np.array(data_y_train) 
  my_y_test = np.array(data_y_test)

  tensor_x_train = torch.Tensor(my_x_train) # transform to torch tensor
  tensor_x_test = torch.Tensor(my_x_test)
  tensor_y_train = torch.Tensor(my_y_train) 
  tensor_y_test = torch.Tensor(my_y_test)

  #tensor_x_train, tensor_x_test, tensor_y_train, tensor_y_test = tensor_x_train.to(DEVICE), tensor_x_test.to(DEVICE), tensor_y_train.to(DEVICE), tensor_y_test.to(DEVICE)

  train_dataset = TensorDataset(tensor_x_train, tensor_y_train)
  test_dataset = TensorDataset(tensor_x_test, tensor_y_test)  

  train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE) 
  test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE) 


  #train_loader = [unsqueeze(i,j) for (i,j) in train_loader]
  #train_loader = [swap(i,j) for (i,j) in train_loader]

  #test_loader = [unsqueeze(i,j) for (i,j) in test_loader]
  #test_loader = [swap(i,j) for (i,j) in test_loader]

  # Checking the dataset
  print('Training Set:\n')
  for image_x, image_y in train_loader: 
      print('Image x batch dimensions:', image_x.size())
      print('Image y batch dimensions:', image_y.size())
  

  # Checking the dataset
  print('\nTesting Set:')
  for image_x, image_y in test_loader:
      print('Image x batch dimensions:', image_x.size())
      print('Image y batch dimensions:', image_y.size())
 

  return train_loader, test_loader

def generate_image_loader(data_x, data_y, TRAIN_SIZE, BATCH_SIZE, SEED):

  data_x_train, data_y_train, data_x_test, data_y_test = split_x_y_list(data_x, data_y, TRAIN_SIZE, SEED)

  ##########################
  ### Dataset
  ##########################

  import torch
  import numpy as np
  from torch.utils.data import TensorDataset, DataLoader

  import random

  my_x_train = np.array(data_x_train)
  my_x_test = np.array(data_x_test)

  my_y_train = np.array(data_y_train) 
  my_y_test = np.array(data_y_test)

  tensor_x_train = torch.Tensor(my_x_train) # transform to torch tensor
  tensor_x_test = torch.Tensor(my_x_test)
  tensor_y_train = torch.Tensor(my_y_train) 
  tensor_y_test = torch.Tensor(my_y_test)

  #tensor_x_train, tensor_x_test, tensor_y_train, tensor_y_test = tensor_x_train.to(DEVICE), tensor_x_test.to(DEVICE), tensor_y_train.to(DEVICE), tensor_y_test.to(DEVICE)

  train_dataset = TensorDataset(tensor_x_train, tensor_y_train)
  test_dataset = TensorDataset(tensor_x_test, tensor_y_test)  

  train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE) 
  test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE) 


  train_loader = [unsqueeze(i,j) for (i,j) in train_loader]
  train_loader = [swap(i,j) for (i,j) in train_loader]

  test_loader = [unsqueeze(i,j) for (i,j) in test_loader]
  test_loader = [swap(i,j) for (i,j) in test_loader]

  # Checking the dataset
  print('Training Set:\n')
  for image_x, image_y in train_loader: 
      print('Image x batch dimensions:', image_x.size())
      print('Image y batch dimensions:', image_y.size())
     

  # Checking the dataset
  print('\nTesting Set:')
  for image_x, image_y in test_loader:
      print('Image x batch dimensions:', image_x.size())
      print('Image y batch dimensions:', image_y.size())
 

  return train_loader, test_loader

##########################
### MODEL
##########################


class Reshape(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.shape = args

    def forward(self, x):
        return x.view(self.shape)


class Trim(nn.Module):
    def __init__(self, *args):
        super().__init__()

    def forward(self, x):
        return x[:, :, :256, :256]


class MLP(nn.Module):

    def __init__(self, n_components_x, n_components_y):
        super().__init__()
        
        self.ANN = nn.Sequential( #784
                nn.Linear(n_components_x, 100),
                nn.LeakyReLU(0.01),
                nn.Linear(100, 100),
                nn.LeakyReLU(0.01),
                nn.Linear(100, 100),
                nn.LeakyReLU(0.01),
                nn.Linear(100, 100),
                nn.LeakyReLU(0.01),
                nn.Linear(100, n_components_y)
                )

    def forward(self, x):
        x = self.ANN(x)
        return x
def run_cae(Rai, Raf, ti, tf, mode, pca_var, TRAIN_SIZE, BATCH_SIZE, SEED, RANDOM_SEED, LEARNING_RATE, NUM_EPOCHS, DEVICE):

  data_x, data_y = load_data(Rai, Raf, ti, tf, mode)

  data_y_ref = data_y.copy()

  data_x, data_y, n_components_x, n_components_y, pca_x, pca_y = principal_components(data_x, data_y, pca_var)

  train_loader, test_loader = generate_loader(data_x, data_y, TRAIN_SIZE, BATCH_SIZE, SEED)  

  word = 'word'
  
  print(n_components_x, n_components_y)


  #time.sleep(30)
  model = MLP(n_components_x, n_components_y)
  model = model.cpu()
  optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
  #test_acc = 0
  del model
  del optimizer
  #del test_acc
  gc.collect()
  torch.cuda.empty_cache()

  #time.sleep(30)


  set_all_seeds(RANDOM_SEED)

  model = MLP(n_components_x, n_components_y)
  model = model.to(DEVICE)

  optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
  
  model = model.cpu()
  del model
  del optimizer
  gc.collect()
  torch.cuda.empty_cache()

  model = MLP(n_components_x, n_components_y)
  model = model.to(DEVICE)

  optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


  log_dict = train_autoencoder_v1(num_epochs=NUM_EPOCHS, model=model, 
                                  optimizer=optimizer,
                                  train_loader=train_loader, device = DEVICE,
                                  skip_epoch_stats=True)
  
  test_acc = compute_test_accuracy(Rai, Raf, ti, tf, mode, TRAIN_SIZE, BATCH_SIZE, SEED, pca_var, model, DEVICE)

  print(test_acc)

  if mode == 'diffusion_mask':
    word = 'diffusion'
  elif mode == 'convection_mask':
    word = 'convection'
  elif mode == 'no_masking':
    word = 'base'

  os.mkdir('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + str(ti) + '_' + str(tf)  + '/models/' + word + '/' + 'PCA_ANN_' + str(n_components_x) + '_' + str(n_components_y) + '_' + str(pca_var) + '_RS' + str(RANDOM_SEED))

  torch.save(model.state_dict(), '/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + str(ti) + '_' + str(tf)  + '/models/' + word + '/' + 'PCA_ANN_' + str(n_components_x) + '_' + str(n_components_y) + '_' + str(pca_var) + '_RS' + str(RANDOM_SEED) + '/model.pt')

  np.save('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + str(ti) + '_' + str(tf)  + '/models/' + word + '/' + 'PCA_ANN_' + str(n_components_x) + '_' + str(n_components_y) + '_' + str(pca_var) + '_RS' + str(RANDOM_SEED) + '/test_acc.npy', test_acc, allow_pickle=True)
  np.save('/content/gdrive/My Drive/Project/Results/' + Rai + '_' + Raf + '_' + str(ti) + '_' + str(tf)  + '/models/' + word + '/' + 'PCA_ANN_' + str(n_components_x) + '_' + str(n_components_y) + '_' + str(pca_var) + '_RS' + str(RANDOM_SEED) + '/log_dict.npy', log_dict, allow_pickle=True)

  print()
  